# 课程大纲：
 - fast.ai 概述
 - 图像识别go through
 - fast.ai小优化

fast.ai 作者曾取得多次Kaggle竞赛第一名，基于自己对ML的理解，以及时下最火热的paper，在PyTorch基础上构建了fast.ai框架，帮助初学者快速构建高准确率的ML应用，先安装必要的软件包：

In [ ]:
!pip install fastai

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

从导入fastai各个模块开始，fast.ai推荐的做法是按照模块导入*，如果科学家不知道导入某个具体的模块，可直接导入 fastai.**


In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

设置batchSize，取决于输入图像的分辨率，模型大小和显卡的缓存大小，如果过大，可适当调小bs值

In [ ]:
#bs = 64
bs = 32   # uncomment this line if you run out of memory even after clicking Kernel->Restart

## 查验数据

本次实验将使用[O. M. Parkhi et al., 2012](http://www.robots.ox.ac.uk/~vgg/publications/2012/parkhi12a/parkhi12a.pdf)中提供的[Oxford-IIIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/)作为数据集，该数据集提供了12类🐱及25类🐶的图片，我们的模型需要学会分辨这37类不同的动物。据论文中的描述，2012年能取得的最好准确率为59.21%, 接下来我们将构建自己的模型，看看能取得怎样的准确率!

我们将使用 `untar_data` 函数来下载并解压数据集，此函数需要一个URL来作为输入参数.

In [ ]:
help(untar_data)

这种写法为的是节约行数，其实是两条语句，先赋值，后打印.
URLs里包含了一些可直接使用的数据集的url，参见：https://github.com/fastai/fastai/blob/master/fastai/datasets.py

In [ ]:
path = untar_data(URLs.PETS); path

In [ ]:
path.ls()

这是Python3里的语法糖，用于构建跨平台的路径，Linux系统下用/来分割路径，Windows系统下用\来分割路径

In [ ]:
path_anno = path/'annotations'
path_img = path/'images'

当我们处理问题时，我们要做的第一件事就是看看数据。 我们总是需要非常清楚地了解问题是什么，以及数据是什么样子，然后我们才能弄清楚如何解决问题。 查看数据意味着了解数据目录的结构、标签是什么以及某些示例图像的样子.

处理不同的图像分类数据集之间的主要区别在于标注的存储方式，比如ImageNet用分类编号作为目录名。 在本实验中使用的数据集中，标签存储在文件名中。 我们需要提取它们，以便能够将图片分类为正确的类别。 幸运的是，fastai 库有一个方便的功能可用于这个目的， `ImageDataBunch.from_name_re` 通过 [正则表达式](https://docs.python.org/3.6/library/re.html)来从文件名获取图片分类.

In [ ]:
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
np.random.seed(2)
pat = r'/([^/]+)_\d+.jpg$'

In [ ]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=224, bs=bs
                                  ).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

data.classes与data.c等效，用于获取数据集的分类数

In [ ]:
print(data.classes)
len(data.classes),data.c

## 模型训练: resnet34

现在我们将开始训练模型，我们将使用 [卷积神经网络] (http://cs231n.github.io/convolutional-networks/) 骨干，单个隐藏层以及全连接作为分类器。fastai库里已包含卷积神经网络，直接使用`cnn_learner`即可构建一个卷积神经网络模型。`cnn_learner`可直接使用torchvision中的模型定义，完全不用自己构建模型（绝大多数情况下也不需要自己构建），参见：https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

训练次数为4个epochs (所有数据的4次循环).



In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

打印模型结构：

In [ ]:
learn.model

## fastai小优化1: fit_one_cycle

One-Cycle-Policy 大概有三个步骤：

我们逐渐将学习率从 lr_max / div_factor 提高到 lr_max，同时我们逐渐减少从 mom_max 到 mom_min 的动量(momentum)。

反向再做一次：我们逐渐将学习率从 lr_max 降低到 lr_max / div_factor，同时我们逐渐增加从 mom_min 到 mom_max 的动量。

我们进一步将学习率从 lr_max / div_factor 降低到 lr_max /（div_factor x 100），我们保持动力稳定在 mom_max。

正是在一个epoch里learning rate就已经不断调整并被观察，所以后续可通过lr_find()函数寻找最佳的lr，不必通过HPO去通过不同的epoch来寻找最优learning rate参数，缩短训练时间，并提高准确度。

这里先简单介绍一下原理，具体如何使用先不展开

In [ ]:
learn.fit_one_cycle(4)

保存模型结构和模型参数，训练完后保存以下模型是一个好习惯，避免后续调整操作影响模型

In [ ]:
learn.save('stage-1')

## 训练结果

让我们来看看结果如何。 

我们将首先看到哪些是模型彼此混淆最多的类别。 我们将尝试看看模型预测是否合理。 在这种情况下，错误看起来合理（没有明显的低级错误）。 这表明我们的分类器工作正常。 

此外，当我们绘制混淆矩阵时，我们可以看到分布严重偏斜：模型一遍又一遍地犯下同样的错误，但很少混淆其他类别。 这表明模型只是很难区分某些特定类别，这是正常的行为。

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

接下来，打印出最容易混淆的图片类别，每张图片上方按照以下格式给出结果： 预测值/真实值/损失/置信度，按照损失值从高到低排列，损失值越高，表明预测结果和真实值的差异越大

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
doc(interp.plot_top_losses)

如何解读混淆矩阵？纵坐标表示真实的分类，横坐标表示模型预测的分类，从左上角到右下角的对角线上的值表示预测正确的数量，其他值都是预测错误的数量，我们可根据其值来分析哪些类别的图片容易被误判，并寻找优化的方法

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

另一种方式，就是通过`most_confused`函数来打印出最容出错的判断

In [ ]:
interp.most_confused(min_val=2)

## 解冻、微调和学习率

既然我们的模型正如我们所期望的那样工作，我们将 *解冻*模型并继续进行训练。

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

加载之前保存的模型：

In [ ]:
learn.load('stage-1');

## 接着讨论fit_one_cycle的收益

经过fit_one_cycle，在一个epoch里已经尝试过了不同的learning rate，系统会记录不同lr下的loss，通过lr_find()函数，整理出不同的lr对应的loss，通过图表可以直观的展示哪一个lr对应的loss最低，再次进行训练时，就可以选择某个区间的lr进一步优化训练结果。

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

这已经是准确度相当高的模型了，接下来我们还可通过替换骨干来构建更复杂的模型，进一步提高准确度!

## 模型训练: resnet50

现在我们将用与以前相同的方式训练，但有一个区别：我们将使用resnet50而不是使用resnet34作为架构，（resnet34 是一个34层的残差网络，而 resnet50有50层。 关于[残差网络](https://arxiv.org/pdf/1512.03385.pdf)可参见论文）。 基本上，resnet50 通常表现更好，因为它是一个具有更多参数的更深的网络。 让我们来看看我们是否可以在这里实现更高的性能。 为了更好的训练，我们会使用较大的图像，因为这样网络可以看到更多细节。 我们减少了一点批处理大小，否则这个较大的网络将需要更多的 GPU 显存。

加载数据，并进行normalize，一个函数搞定。具体imagenet_stats参见：https://github.com/fastai/fastai/blob/3d6d17250cce719cb77d274b3f723d4295df07fd/fastai/vision/data.py

In [ ]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(),
                                   size=299, bs=bs//2).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.save('stage-1-50')

令人惊讶的是，能够如此准确地识别宠物品种！ 让我们看看完全微调是否有帮助：

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-6,1e-4))

如果没有，您可以随时返回到以前的模型。这也就是之前为什么推荐及时保存模型的原因。

In [ ]:
learn.load('stage-1-50');

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.most_confused(min_val=2)

## 其他数据集的使用

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE); path

In [ ]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_folder(path, ds_tfms=tfms, size=26)

In [ ]:
data.show_batch(rows=3, figsize=(5,5))

In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=accuracy)
learn.fit(2)

In [ ]:
df = pd.read_csv(path/'labels.csv')
df.head()

In [ ]:
data = ImageDataBunch.from_csv(path, ds_tfms=tfms, size=28)

In [ ]:
data.show_batch(rows=3, figsize=(5,5))
data.classes

In [ ]:
data = ImageDataBunch.from_df(path, df, ds_tfms=tfms, size=24)
data.classes

In [ ]:
fn_paths = [path/name for name in df['name']]; fn_paths[:2]

In [ ]:
pat = r"/(\d)/\d+\.png$"
data = ImageDataBunch.from_name_re(path, fn_paths, pat=pat, ds_tfms=tfms, size=24)
data.classes

In [ ]:
data = ImageDataBunch.from_name_func(path, fn_paths, ds_tfms=tfms, size=24,
        label_func = lambda x: '3' if '/3/' in str(x) else '7')
data.classes

In [ ]:
labels = [('3' if '/3/' in str(x) else '7') for x in fn_paths]
labels[:5]

In [ ]:
data = ImageDataBunch.from_lists(path, fn_paths, labels=labels, ds_tfms=tfms, size=24)
data.classes

## 额外的bonus

Data Augmentation 数据增强，对已有的数据进行转换和微调，产生更多的相似数据。一方面解决了数据不够丰富的问题，另一方面，也可以对现有数据进行泛化，一方面能很大程度解决overfitting的问题，另一方面也可以提到模型训练的准确度。对于不同类型的数据有一些常见的data augmentation的方法，例如对图片数据，可以进行翻转、旋转、亮度调节、锐度调节、截取、填充等方式来进行图像增强。

![png](data_augmentation.png)

![jpeg](da.jpeg)

对图像的裁剪、旋转是常见的data augmentation方法，可参考https://docs.fast.ai/vision.transform.html 还有一种方式，将一张猫🐱的图片和一张狗🐶的图片混合在一起，那么结果是猫还是🐶？

fast.ai通过mixup方式实现了这种数据增强的方式，那么，什么是mixup？

## What is mixup?
This module contains the implementation of a data augmentation technique called mixup. It is extremely efficient at regularizing models in computer vision (we used it to get our time to train CIFAR10 to 94% on one GPU to 6 minutes).

As the name kind of suggests, the authors of the mixup article propose training the model on mixes of the training set images. For example, suppose we’re training on CIFAR10. Instead of feeding the model the raw images, we take two images (not necessarily from the same class) and make a linear combination of them: in terms of tensors, we have:

new_image = t * image1 + (1-t) * image2

where t is a float between 0 and 1. The target we assign to that new image is the same combination of the original targets:

new_target = t * target1 + (1-t) * target2

assuming the targets are one-hot encoded (which isn’t the case in PyTorch usually). And it's as simple as that.

![png](mixup.png)

Dog or cat? The right answer here is 70% dog and 30% cat!

As the picture above shows, it’s a bit hard for the human eye to make sense of images obtained in this way (although we do see the shapes of a dog and a cat). However, it somehow makes a lot of sense to the model, which trains more efficiently. One important side note is that when training with mixup, the final loss (training or validation) will be higher than when training without it, even when the accuracy is far better: a model trained like this will make predictions that are a bit less confident.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.callbacks.mixup import *
from fastai.vision import *

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy])

In [ ]:
learn.fit(8)

In [ ]:
model = simple_cnn((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy]).mixup()
learner.fit(8)